In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import scipy.stats as st
import sklearn 
import sklearn.datasets
import sklearn.metrics as metrics 
import matplotlib.pyplot as plt 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.feature_selection import mutual_info_classif
from scipy import stats
import collections
import re



C:\Anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Anaconda\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

In [3]:
data2017 = pd.read_excel("PERM_Disclosure_Data_FY17.xlsx")
data2016 = pd.read_excel("PERM_Disclosure_Data_FY16.xlsx")
data2015 = pd.read_excel("PERM_Disclosure_Data_FY15_Q4.xlsx")

In [4]:
#Change the column names so they match between years
data2016 = data2016.rename(columns = {'FW_INFO_REL_OCCUP_EXPERIENCE':'FW_INFO_REL_OCCUP_EXP'})
data2015 = data2015.rename(columns = {'JI_FOREIGN_WORKER_LIVE_ON_PREMISES':'JI_FW_LIVE_ON_PREMISES'})
data2015 = data2015.rename(columns = {'REC_INFO_BARG_REP_NOTIFIED':'RECR_INFO_BARG_REP_NOTIFIED'})
data2015 = data2015.rename(columns = {'JI_OFFERED_TO_SEC_J_FOREIGN_WORKER':'JI_OFFERED_TO_SEC_J_FW'})
data2017 = data2017.rename(columns = {'PW_JOB_TITLE_9089':'PW_Job_Title_9089'})

In [5]:
dataList = [data2017, data2016, data2015]
originalData = pd.concat(dataList)

In [47]:
df = originalData.copy()

In [48]:
#DROP WITHRAWN FROM CASE_STATUS
df = df[df['CASE_STATUS'] != 'Withdrawn']

COLUMNS TO KEEP

In [49]:
df = df[['CASE_STATUS', 'REFILE','FW_OWNERSHIP_INTEREST'
         ,'JOB_INFO_TRAINING','JOB_INFO_EXPERIENCE','JOB_INFO_FOREIGN_LANG_REQ'
         ,'JI_OFFERED_TO_SEC_J_FW','RECR_INFO_COLL_TEACH_COMP_PROC'
         ,'RI_COLL_TCH_BASIC_PROCESS','RECR_INFO_SUNDAY_NEWSPAPER','RECR_INFO_EMPLOYER_REC_PAYMENT'
         ,'RI_POSTED_NOTICE_AT_WORKSITE','RI_LAYOFF_IN_PAST_SIX_MONTHS','RI_US_WORKERS_CONSIDERED'
         ,'FW_INFO_TRAINING_COMP','FW_INFO_REQ_EXPERIENCE','FW_INFO_ALT_EDU_EXPERIENCE'
         ,'FW_INFO_REL_OCCUP_EXP','PREPARER_INFO_EMP_COMPLETED','RECR_INFO_BARG_REP_NOTIFIED', 'COUNTRY_OF_CITIZENSHIP']]

In [50]:
Countries = pd.read_excel("Countries.xlsx")

In [51]:
CountriesDf = Countries.copy()

In [52]:
CountriesDf

,Region,Country
0,1,Unknown
1,13,AFGHANISTAN
2,17,ALAND
3,18,ALBANIA
4,4,ALGERIA
5,23,AMERICAN SAMOA
6,18,ANDORRA
7,3,ANGOLA
8,7,ANGUILLA
9,1,ANTARCTICA


In [53]:
dfList = df['COUNTRY_OF_CITIZENSHIP'].tolist()
CountriesList = CountriesDf['Country'].tolist()

In [54]:
count = 0
regionList = []
for i in dfList:
    count += 1
    if(count%10000 == 0):
        print(count)
    country = i
    country = str(country)
    countryList = country.split(" AND ")
    found = False
    for j, k in zip(CountriesList, RegionsList):
        if(country == j):
            regionList.append(k)
            found = True
            break;
        elif(any(j in s for s in countryList)):
            regionList.append(k)
            found = True
            break;
    if(found != True):
        regionList.append(1)

CitizenLoc = df.columns.get_loc('COUNTRY_OF_CITIZENSHIP')
df.insert(CitizenLoc + 1, 'CitizenRegion',value = regionList)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000


In [55]:
print(list(zip(regionList, dfList))[:500])

[(1, 'TAIWAN'), (9, 'VENEZUELA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'NEPAL'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (17, 'SWEDEN'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (10, 'CANADA'), (13, 'INDIA'), (8, 'MEXICO'), (12, 'CHINA'), (13, 'INDIA'), (15, 'JORDAN'), (13, 'INDIA'), (12, 'SOUTH KOREA'), (13, 'INDIA'), (16, 'RUSSIA'), (9, 'PERU'), (7, 'BAHAMAS'), (13, 'INDIA'), (12, 'MONGOLIA'), (13, 'INDIA'), (13, 'INDIA'), (12, 'CHINA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (12, 'CHINA'), (16, 'UKRAINE'), (17, 'UNITED KINGDOM'), (13, 'INDIA'), (13, 'INDIA'), (10, 'CANADA'), (14, 'PHILIPPINES'), (14, 'PHILIPPINES'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), (13, 'INDIA'), 

In [56]:
df.tail(50)

,CASE_STATUS,REFILE,FW_OWNERSHIP_INTEREST,JOB_INFO_TRAINING,JOB_INFO_EXPERIENCE,JOB_INFO_FOREIGN_LANG_REQ,JI_OFFERED_TO_SEC_J_FW,RECR_INFO_COLL_TEACH_COMP_PROC,RI_COLL_TCH_BASIC_PROCESS,RECR_INFO_SUNDAY_NEWSPAPER,RECR_INFO_EMPLOYER_REC_PAYMENT,RI_POSTED_NOTICE_AT_WORKSITE,RI_LAYOFF_IN_PAST_SIX_MONTHS,RI_US_WORKERS_CONSIDERED,FW_INFO_TRAINING_COMP,FW_INFO_REQ_EXPERIENCE,FW_INFO_ALT_EDU_EXPERIENCE,FW_INFO_REL_OCCUP_EXP,PREPARER_INFO_EMP_COMPLETED,RECR_INFO_BARG_REP_NOTIFIED,COUNTRY_OF_CITIZENSHIP,CitizenRegion
84886,Denied,Y,NaN,N,N,N,Y,N,N,Y,N,A,N,A,NaN,NaN,NaN,NaN,N,A,MEXICO,8
84887,Denied,N,NaN,N,N,N,Y,N,N,Y,N,Y,N,N,NaN,NaN,NaN,NaN,N,A,SOUTH KOREA,12
84888,Denied,N,NaN,N,N,N,Y,N,N,Y,N,Y,N,N,NaN,NaN,NaN,NaN,N,A,SOUTH KOREA,12
84889,Denied,N,N,N,N,N,Y,NaN,NaN,Y,N,Y,N,A,A,A,A,A,N,A,MEXICO,8
84890,Denied,N,N,N,Y,N,Y,N,Y,Y,N,Y,N,A,A,Y,A,A,N,Y,VENEZUELA,9
84891,Denied,N,N,N,Y,N,Y,N,Y,Y,N,Y,N,A,A,Y,A,A,N,Y,COLOMBIA,9
84892,Denied,N,N,N,N,N,Y,NaN,NaN,Y,N,Y,N,NaN,A,A,A,A,Y,A,VIETNAM,1
84893,Denied,N,N,N,N,N,Y,NaN,NaN,Y,N,Y,N,NaN,A,A,A,A,Y,A,CHINA,12
84894,Denied,N,N,N,N,N,Y,NaN,NaN,Y,N,A,N,NaN,A,A,A,A,Y,Y,SOUTH KOREA,12
84895,Denied,N,N,N,N,N,Y,NaN,NaN,Y,N,A,N,NaN,A,A,A,A,Y,Y,SOUTH KOREA,12


In [43]:
(df['CitizenRegion'].value_counts(dropna=False) / df['CitizenRegion'].value_counts(dropna=False).sum()) * 100

13    57.695930
12    16.494204
10     3.716413
9      3.037434
14     2.772832
8      2.767839
1      2.255609
16     1.974032
15     1.904137
17     1.882503
19     1.752366
18     1.376597
20     0.488266
6      0.425693
7      0.425361
4      0.394407
2      0.274920
5      0.204692
11     0.105508
3      0.043934
21     0.004992
22     0.001997
23     0.000333
Name: CitizenRegion, dtype: float64